<a href="https://colab.research.google.com/github/0xZee/groq-tavily-bot/blob/main/tavily_li.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`LLAMA-INDEX, TAVILY `: Enhanced Search Bot

search: Search for relevant dynamic data based on a query. Returns a list of urls and their relevant content.
This loader is designed to be used as a way to load data as a Tool in an Agent.

INSTALL & LIBRARIES
---

In [ ]:
!pip install -q llama-index llama-hub tavily-python llama-index-llms-groq
!pip install -q llama-index-embeddings-cohere
!pip install transformers -U

In [2]:
from google.colab import userdata
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.schema import Document
from llama_index.llms.groq import Groq
from llama_index.embeddings.cohere import CohereEmbedding
# tavily
from tavily import TavilyClient
#from llama_hub.tools.tavily_research import TavilyToolSpec

Tavily Data 🐳 : CODE #1
---

In [3]:
# Tavily Context Search function -> Document
def get_Document(query):
  # Initialize the Tavily client with your API key
  tavily = TavilyClient(api_key=userdata.get('TAVILY_API'))
  response = tavily.search(query=query, search_depth="advanced")

  return [
      Document(text=result["content"], extra_info={"url": result["url"]})
      for result in response["results"]
  ]

In [4]:
# Get Document for Index
query = "What is the risks, headwinds and tailwinds of investing in AAPL this year, also give the last stock price and market cap ?"
documents = get_Document(query)

documents[0].text

"(Definition, Examples, and Applications)\nSelf-Employment: Benefits, Challenges, Financial Planning\nTop Resources\nInvestment Banking Salary\nPrivate Equity Salary\nInvestment Banking Target School List\nCompany\nsupport@peakframeworks.com\n🙌\xa0 Give us a Shout Out!\n🌳\xa0 We've planted 31,504 trees\nPrivate Equity Recruiting Guide\nInvestment Banking Coffee Chat Questions Courses\nReviews\nAbout Us\nBlog\nLogin\nMore\nWhat are Headwinds vs. Tailwinds in Business?\nHeadwinds refer to obstacles or challenges that hinder growth or progress, whereas tailwinds are factors that propel something forward.\n For instance, stricter environmental regulations might push companies towards greener initiatives but also impose additional costs.\nFactors that Lead to Headwinds\nEconomic Downturns or Recessions\nEconomic downturns, characterized by declining GDP and an increase in unemployment, can create severe headwinds for a variety of industries.\n For instance, early investors in the electric v

In [ ]:
# Setting llm and embeddings
GROQ_API_KEY = userdata.get('GROQ_API_KEY') # Models: "llama3-8b-8192" ; "gemma-7b-it" ; "mixtral-8x7b-32768"
llm_groq = Groq(model="llama3-8b-8192", api_key=GROQ_API_KEY)
Settings.llm = llm_groq
embed_co = CohereEmbedding(cohere_api_key=userdata.get('COHERE_API'), model_name="embed-english-v3.0", input_type="search_query")
Settings.embed_model = embed_co

#query = "As financial adviser, What is the risks and tailwinds of investing in AI infra stocks in 2024, also give some stocks names with details ?"
query = "tell me more aboout OKLO and focus in financial situation, industry, strategy, last news and give some competitors"
#query = "Give me in a table format the first 11 Cypto Coin by market cap in dollars, with price and brief description  ?"

#doc_store = SimpleDirectoryReader('data').load_data()
documents = get_Document(query)
index = VectorStoreIndex.from_documents(documents)

# Set ChatEngin 'Context' Mode
chat_engine = index.as_chat_engine(chat_mode="context" , verbose=True)


In [ ]:
# Stream response
response = chat_engine.stream_chat(query)
for token in response.response_gen:
    print(token, end="")

Here's an overview of Oklo Inc. (OKLO) and its financial situation, industry, strategy, last news, and competitors:

**Financial Situation:**

* Oklo Inc. is a private company, so its financial information is not publicly disclosed. However, according to PitchBook, Oklo has received funding from investors, including a $50 million Series B round in 2022.
* The company's financial situation is likely focused on developing its advanced fission power plants and scaling its operations.

**Industry:**

* Oklo operates in the nuclear energy industry, specifically in the development of advanced fission power plants.
* The company's technology aims to provide clean, reliable, and affordable energy at scale.

**Strategy:**

* Oklo is pursuing two complementary tracks to address the demand for clean energy:
	1. Providing reliable, commercial-scale energy to customers.
	2. Selling used nuclear fuel recycling services to the U.S. market.
* The company's strategy is focused on developing its advance

Tavily Data 🐳 : CODE #2
---

In [ ]:
from google.colab import userdata
from tavily import TavilyClient

tavily = TavilyClient(api_key=userdata.get('TAVILY_API'))

# For basic search:
#response = tavily.search(query="Get the main data of PLTR and Should I invest in PLTR in 2024?")
# For advanced search:

def format_query(symbol):
   return f"Should I invest in {symbol} in 2024?"

symbol = "BBAI"

response = tavily.search(query=format_query(symbol), search_depth="advanced")
# Get the search results as context to pass an LLM:
context = [{"url": obj["url"], "content": obj["content"]} for obj in response['results']]

print(response)
print(len(context))
print(context[0])
print(context[0]['url'])


{'query': 'Should I invest in BBAI in 2024?', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': 'Should You Invest in BigBear.ai (BBAI) Ahead of Q1 Earnings?', 'url': 'https://www.zacks.com/stock/news/2265309/should-you-invest-in-bigbearai-bbai-ahead-of-q1-earnings', 'content': 'BigBear.ai (BBAI Quick Quote BBAI - Free Report) is set to report first-quarter 2024 results on May 2. BigBear.ai is likely to have benefited from growing government investment in its robust AI ...', 'score': 0.97835, 'raw_content': None}, {'title': 'BigBear.ai Stock: Buy, Sell, or Hold? | The Motley Fool', 'url': 'https://www.fool.com/investing/2024/06/08/bigbearai-stock-buy-sell-or-hold/', 'content': 'Shares of BigBear.ai (BBAI 5.88%) have been treading water since a less-than-stellar first-quarter update was released last month. The stock trades down nearly 34% so far in 2024 with the market ...', 'score': 0.97262, 'raw_content': None}, {'title': 'Should You Invest in BigBear

In [ ]:
tavily.qna_search(query='Should I invest in PLTR in 2024?')

'Based on the data available as of June 7, 2024, Palantir Technologies (PLTR) stock has been experiencing volatility and fluctuation in price. The stock price has seen significant increases over the past year, with forecasts indicating potential growth. However, there are concerns about meeting consensus expectations for revenue growth in 2024, particularly in the international commercial business and government sectors. Analysts have highlighted challenges related to government contracts and competition in data analytics. Therefore, investing in PLTR in 2024 may involve risks and it is advisable to carefully consider all factors before making a decision.'

In [ ]:
resp = tavily.get_search_context(query="What happened in the burning man floods?", search_depth="advanced", max_tokens=1500)

In [ ]:
resp

'"[\\"{\\\\\\"url\\\\\\": \\\\\\"https://www.today.com/news/what-is-burning-man-flood-death-rcna103231\\\\\\", \\\\\\"content\\\\\\": \\\\\\"\\\\\\\\\\\\\\"\\\\\\\\nAn update on BurningMan.org posted the night of Sept. 3 offered those still on site a \\\\\\\\\\\\\\"Wet Playa Survival Guide\\\\\\\\\\\\\\" with tips, as well as an update saying that conditions were \\\\\\\\\\\\\\"improving\\\\\\\\\\\\\\" and that roads in Black Rock City were expected to open on Sept. 4 for \\\\\\\\\\\\\\"Exodus.\\\\\\\\\\\\\\"\\\\\\\\n\\\\\\\\\\\\\\"Thank goodness this community knows how to take care of each other,\\\\\\\\\\\\\\" the Instagram page for Burning Man Information Radio wrote on a post predicting more rain.\\\\\\\\nNeal Katyal, Chris Rock and others detail their \\\\\\\\u2018escape\\\\\\\\u2019 from Black Rock City\\\\\\\\nComedian Chris Rock and Diplo, producer and DJ,\\\\\\\\u00a0were among this year\'s Burning Man attendees. Why are people stranded at Burning Man?\\\\\\\\nEach year parti

In [ ]:
for item in context :
  print(item['content'])

"
The massive number of installed devices worldwide helps Apple make more money through its services, such as the App Store, Apple Music, and Apple Pay. He has said that the introduction of the product "marks the beginning of a new era for computing" and that Vision Pro is "unlike anything created before. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, top-rated podcasts, and non-profit The Motley Fool Foundation.
 The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, top-rated podcasts, and non-profit The Motley Fool Foundation.
 "
Apple's downsides
As great as Apple is, the company and its stock have some downsides.
Investing
Tony Dong
Gold price today November 15, 2023
Investing
Tony Dong
Microsoft (MSFT) stock forecast and price prediction
Investing
Wayne Duggan
Gold price today November 14, 2023
Inve

In [ ]:
import os

# Assuming 'context' is your list of dictionaries with content
for i, item in enumerate(context):
    # Create the 'data' directory if it doesn't exist
    if not os.path.exists(f"data"):
        os.makedirs(f"data")

    # Define the filename using the current index
    filename = f"data/context_{symbol}_{i}.txt"

    # Write the content to the file
    with open(filename, 'w') as file:
        file.write(item['content'])


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.cohere import CohereEmbedding

In [ ]:
embed_co = CohereEmbedding(cohere_api_key=userdata.get('COHERE_API'), model_name="embed-english-v3.0", input_type="search_query")
Settings.embed_model = embed_co
doc_store = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(doc_store)

print(len(doc_store))
print(doc_store[0])

10
Doc ID: 6581e989-5f11-44f1-8e21-bff528d5c92a
Text: BigBear.ai BBAI is set to report first-quarter 2024 results on
May 2.BigBear.ai is likely to have benefited from growing government
investment in its robust AI solutions.BBAI's ongoing contracts ...


xxx
---

xxx
---